In [1]:
import torch
import numpy as np

In [2]:
root_path = "mpi3d"
labels = np.load(f"{root_path}/mpi3d_composition_train_labels.npz")

In [3]:
# open original dataset

print("Loading file:")
data = np.load(f"{root_path}/mpi3d_real.npz", mmap_mode='r')['images']
print("To PyTorch")
data = torch.from_numpy(np.array(data))  # this will trigger actual load, but only when needed
print("View")
data = data.view(6, 6, 2, 3, 3, 40, 40, 64, 64, 3)


Loading file:
To PyTorch
View


In [4]:
data = data.permute(0,1,2,3,4,5,6,9,7,8)
#data = data.view(6, 6, 2, 3, 3, 40, 40, 3, 64, 64)



In [5]:
print(data.shape)

torch.Size([6, 6, 2, 3, 3, 40, 40, 3, 64, 64])


In [6]:
# Step 2: Create latent IDs by computing the grid of all combinations
images = data.reshape(-1, 3, 64, 64)  # shape: (1_555_200, 64, 64, 3)
latent_shapes = data.shape[:-3]  # (6, 6, 2, 3, 3, 40, 40)
latent_dims = [torch.arange(s) for s in latent_shapes]
mesh = torch.meshgrid(*latent_dims, indexing='ij')
latent_ids = torch.stack([g.flatten() for g in mesh], dim=1)  # shape: (1_555_200, 7)

# Check sizes
print("Images shape:", images.shape)
print("Latent IDs shape:", latent_ids.shape)


Images shape: torch.Size([1036800, 3, 64, 64])
Latent IDs shape: torch.Size([1036800, 7])


In [ ]:
new_data = {"images": images, 'latent_ids': latent_ids}
torch.save(new_data, f"{root_path}/mpi3d.pth")

In [10]:
torch.load(f"3dshapes/3dshapes.pth")['latent_ids'].shape

torch.Size([480000, 6])

In [34]:
root_path = "mpi3d"
# maps latent vector to index in full dataset
multiplier = torch.tensor([6*2*3*3*40*40, 2*3*3*40*40, 3*3*40*40, 3*40*40, 40*40, 40, 1]).unsqueeze(1)

for sub_dataset in ['interpolation', 'extrapolation', 'composition']:
    for split in ['train','test']:
        labels =  np.load(f"{root_path}/mpi3d_{sub_dataset}_{split}_labels.npz")['arr_0']
        labels = torch.from_numpy(labels)
        indices = torch.matmul(labels, multiplier).squeeze()
        torch.save(indices, f"{root_path}/mpi3d_{sub_dataset}_{split}_indices.pth")
        



In [13]:
latent_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0])

In [18]:
import torch
root_path = "mpi3d"
sub_dataset = "composition"
split = "train"
indices = torch.load(f"{root_path}/{root_path}_{sub_dataset}_{split}_indices.pth")

In [19]:
indices.shape

torch.Size([287712])

In [17]:
labels =  np.load(f"{root_path}/mpi3d_{sub_dataset}_{split}_labels.npz")['arr_0']

In [25]:
labels[250000]

array([ 5,  1,  0,  1,  2,  0, 28])

In [26]:
latent_ids[indices[250000]]

tensor([[ 5,  1,  0,  1,  0,  6, 28]])

In [5]:
for n,(i,j) in enumerate(zip(labels, indices)):
    print(i, j)
    if n>100:
        break

ERROR! Session/line number was not unique in database. History logging moved to new session 1004
tensor([0, 0, 0, 0, 0, 6, 6]) tensor([246])
tensor([0, 0, 0, 0, 0, 6, 7]) tensor([247])
tensor([0, 0, 0, 0, 0, 6, 8]) tensor([248])
tensor([0, 0, 0, 0, 0, 6, 9]) tensor([249])
tensor([ 0,  0,  0,  0,  0,  6, 10]) tensor([250])
tensor([ 0,  0,  0,  0,  0,  6, 11]) tensor([251])
tensor([ 0,  0,  0,  0,  0,  6, 12]) tensor([252])
tensor([ 0,  0,  0,  0,  0,  6, 13]) tensor([253])
tensor([ 0,  0,  0,  0,  0,  6, 14]) tensor([254])
tensor([ 0,  0,  0,  0,  0,  6, 15]) tensor([255])
tensor([ 0,  0,  0,  0,  0,  6, 16]) tensor([256])
tensor([ 0,  0,  0,  0,  0,  6, 17]) tensor([257])
tensor([ 0,  0,  0,  0,  0,  6, 18]) tensor([258])
tensor([ 0,  0,  0,  0,  0,  6, 19]) tensor([259])
tensor([ 0,  0,  0,  0,  0,  6, 20]) tensor([260])
tensor([ 0,  0,  0,  0,  0,  6, 21]) tensor([261])
tensor([ 0,  0,  0,  0,  0,  6, 22]) tensor([262])
tensor([ 0,  0,  0,  0,  0,  6, 23]) tensor([263])
tensor([ 0,  

In [20]:
data['latent_ids']

TypeError: new(): invalid data type 'str'

In [21]:
data.shape

torch.Size([6, 6, 2, 3, 3, 40, 40, 3, 64, 64])

In [25]:
latent_ids.unique(dim=0).shape

torch.Size([1036800, 7])

In [26]:
latent_ids.shape

torch.Size([1036800, 7])

In [27]:
latent_to_idx = {
    tuple(l.tolist()): idx for idx, l in enumerate(latent_ids)
}

In [28]:
len(latent_to_idx)

1036800

In [35]:
indices.unique().shape

torch.Size([749088])

In [1]:
latent_dims = [6, 6, 2, 3, 3, 40, 40]  # max values per latent dim (exclusive)
n_latents = len(latent_dims)
max_delta = 3  # max distance to try (you can change this)

sample_latent = [5, 0, 1, 2, 1, 10, 39]  # example starting point


def generate_deltas(latent, latent_dims, max_delta):
    for dim in reversed(range(len(latent_dims))):  # iterate from last to first dim
        for delta_val in range(1, max_delta):
            for sign in [1, -1]:
                new_val = latent[dim] + sign * delta_val
                if 0 <= new_val < latent_dims[dim]:
                    delta = [0] * len(latent_dims)
                    delta[dim] = sign * delta_val
                    yield tuple(delta)


print("Latent:", sample_latent)
print("Valid deltas:")
for d in generate_deltas(sample_latent, latent_dims, max_delta):
    print(d)

Latent: [5, 0, 1, 2, 1, 10, 39]
Valid deltas:
(0, 0, 0, 0, 0, 0, -1)
(0, 0, 0, 0, 0, 0, -2)
(0, 0, 0, 0, 0, 1, 0)
(0, 0, 0, 0, 0, -1, 0)
(0, 0, 0, 0, 0, 2, 0)
(0, 0, 0, 0, 0, -2, 0)
(0, 0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, -1, 0, 0)
(0, 0, 0, -1, 0, 0, 0)
(0, 0, 0, -2, 0, 0, 0)
(0, 0, -1, 0, 0, 0, 0)
(0, 1, 0, 0, 0, 0, 0)
(0, 2, 0, 0, 0, 0, 0)
(-1, 0, 0, 0, 0, 0, 0)
(-2, 0, 0, 0, 0, 0, 0)


In [2]:
import itertools

# Latent space settings
latent_dims = [6, 6, 2, 3, 3, 40, 40]  # Valid values: 0 to dim[i]-1
sample_latent = [5, 0, 1, 2, 1, 10, 39]
max_delta = 3  # max total L1 distance

# Generator that yields valid deltas ordered by increasing L1 distance
def generate_deltas(latent, latent_dims, max_delta):
    n_latents = len(latent_dims)
    for total_change in range(1, max_delta + 1):  # 1, 2, ..., max_delta
        for deltas in itertools.product(range(-total_change, total_change + 1), repeat=n_latents):
            if sum(abs(d) for d in deltas) != total_change:
                continue  # skip if not exact L1 distance
            if deltas == (0,) * n_latents:
                continue  # skip zero delta

            # Check bounds
            new_latent = [l + d for l, d in zip(latent, deltas)]
            if all(0 <= val < maxval for val, maxval in zip(new_latent, latent_dims)):
                yield deltas

# Run and print results
print("Latent:", sample_latent)
print("Deltas (by L1 distance):")
for i, delta in enumerate(generate_deltas(sample_latent, latent_dims, max_delta)):
    print(f"{i}: {delta}")


Latent: [5, 0, 1, 2, 1, 10, 39]
Deltas (by L1 distance):
0: (-1, 0, 0, 0, 0, 0, 0)
1: (0, 0, -1, 0, 0, 0, 0)
2: (0, 0, 0, -1, 0, 0, 0)
3: (0, 0, 0, 0, -1, 0, 0)
4: (0, 0, 0, 0, 0, -1, 0)
5: (0, 0, 0, 0, 0, 0, -1)
6: (0, 0, 0, 0, 0, 1, 0)
7: (0, 0, 0, 0, 1, 0, 0)
8: (0, 1, 0, 0, 0, 0, 0)
9: (-2, 0, 0, 0, 0, 0, 0)
10: (-1, 0, -1, 0, 0, 0, 0)
11: (-1, 0, 0, -1, 0, 0, 0)
12: (-1, 0, 0, 0, -1, 0, 0)
13: (-1, 0, 0, 0, 0, -1, 0)
14: (-1, 0, 0, 0, 0, 0, -1)
15: (-1, 0, 0, 0, 0, 1, 0)
16: (-1, 0, 0, 0, 1, 0, 0)
17: (-1, 1, 0, 0, 0, 0, 0)
18: (0, 0, -1, -1, 0, 0, 0)
19: (0, 0, -1, 0, -1, 0, 0)
20: (0, 0, -1, 0, 0, -1, 0)
21: (0, 0, -1, 0, 0, 0, -1)
22: (0, 0, -1, 0, 0, 1, 0)
23: (0, 0, -1, 0, 1, 0, 0)
24: (0, 0, 0, -2, 0, 0, 0)
25: (0, 0, 0, -1, -1, 0, 0)
26: (0, 0, 0, -1, 0, -1, 0)
27: (0, 0, 0, -1, 0, 0, -1)
28: (0, 0, 0, -1, 0, 1, 0)
29: (0, 0, 0, -1, 1, 0, 0)
30: (0, 0, 0, 0, -1, -1, 0)
31: (0, 0, 0, 0, -1, 0, -1)
32: (0, 0, 0, 0, -1, 1, 0)
33: (0, 0, 0, 0, 0, -2, 0)
34: (0, 0, 0, 0, 0, -1, 